In [1]:
# set cwd and solver
import os
import cobra
cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

In [2]:
# load RPE, PR, Recon3D, and basis Human1 models from cs mods folder
from pathlib import Path
from cobra.io import read_sbml_model, load_matlab_model
mod = read_sbml_model(Path().cwd() / 'cs_mods/July2023' / 'Human-GEM_28062023.xml' )
mod_RPE = read_sbml_model(Path().cwd() /  'cs_mods/July2023' / 'model_1_RPE__VoigtEtAl2019_.xml')
mod_PRrod = read_sbml_model(Path().cwd() /  'cs_mods/July2023' / 'model_2_RodPhotoreceptors__LiangEtAl_2019_.xml' )
mod_PRcone = read_sbml_model(Path().cwd() /  'cs_mods/July2023' / 'model_3_ConePhotoreceptors__LiangEtAl_2019_.xml' )
mod_PRrod_Lukowski = read_sbml_model(Path().cwd() /  'cs_mods/July2023' / 'model_1_AverageRodPRC0,C1,C2,C3,C4,C7(%).xml' )
mod_PRrod_Lukowski.name = 'AverageRodPR_Lukowski' 
mod_PRrod_Lukowski.id = 'AverageRodPR_Lukowski' 
mod_PRcone_Lukowski = read_sbml_model(Path().cwd() /  'cs_mods/July2023' / 'model_7_C10ConePR(%).xml' )
mod_PRcone_Lukowski.name = 'C10ConePR_Lukowski' 
mod_PRcone_Lukowski.id = 'C10ConePR_Lukowski' 

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-03


'' is not a valid SBML 'SId'.
'' is not a valid SBML 'SId'.


In [3]:
mod_PRrod

Name,RodPhotoreceptors__LiangEtAl_2019_
Memory address,14989fa6740
Number of metabolites,3998
Number of reactions,5259
Number of genes,1969
Number of groups,149
Objective expression,1.0*MAR13082 - 1.0*MAR13082_reverse_11d67
Compartments,"Cytosol, Endoplasmic reticulum, Extracellular, Mitochondria, Peroxisome, Golgi apparatus, Lysosome, Nucleus, Inner mitochondria"


import os
import pickle

def pickle_object(obj, path):
    """
    Function to pickle an object and save it to a file
    
    Args:
    obj: the object to be pickled
    path: the path where the pickled object will be saved
    """
    # check if directory exists, if not - create it
    directory = os.path.dirname(path)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    try:
        with open(path, 'wb') as f:
            pickle.dump(obj, f)
        print(f"Object saved to {path}")
    except Exception as e:
        print(f"Failed to pickle object: {e}")

# Use the function
pickle_object(mod_PRcone_Lukowski, 'cs_mods/July2023/mod_PRcone_Lukowski.pickle')

In [4]:
# modify context specific models
from src.modify_model import add_all_EX_rxns, remove_compartment

for m in [mod_RPE, mod_PRcone, mod_PRrod, mod_PRrod_Lukowski, mod_PRcone_Lukowski]: # add ATP DM hydrolysis and lost EX rxns to context specific models
    m.add_reactions([mod.reactions.get_by_id('MAR03964').copy()]) # make sure it contains ATP hydrolysis rxn
    m = add_all_EX_rxns(mod,m) # add lost EX rxns
    for r in [r for r in m.reactions if len(r.products)==0]: 
        r.bounds = (0,1000) # close lb for exchange reactions

In [5]:
# remove nucleus ('n') from photoreceptor models     
mod_PRcone = remove_compartment(mod_PRcone.copy(), 'n')  # remove nucleus ('n') from PR model
mod_PRrod = remove_compartment(mod_PRrod.copy(), 'n')  # remove nucleus ('n') from PR model
mod_PRrod_Lukowski = remove_compartment(mod_PRrod_Lukowski.copy(), 'n')  # remove nucleus ('n') from PR model
mod_PRcone_Lukowski = remove_compartment(mod_PRcone_Lukowski.copy(), 'n')  # remove nucleus ('n') from PR model

Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpdvrv8290.lp
Reading time = 0.02 seconds
: 5149 rows, 13072 columns, 49224 nonzeros


c:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp_72plaae.lp
Reading time = 0.02 seconds
: 5075 rows, 12756 columns, 47716 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpx_w5uv3f.lp
Reading time = 0.02 seconds
: 5267 rows, 13206 columns, 49254 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp00bmnxyn.lp
Reading time = 0.03 seconds
: 5287 rows, 12948 columns, 47916 nonzeros


In [6]:
    from src.modify_model import  add_id_suffix

    # add suffix to all ids in RPE and PR models
    mod_PRrod = add_id_suffix(mod_PRrod, '_PR')
    mod_PRcone = add_id_suffix(mod_PRcone, '_PR')
    mod_PRcone_Lukowski = add_id_suffix(mod_PRcone_Lukowski, '_PR')
    mod_PRrod_Lukowski = add_id_suffix(mod_PRrod_Lukowski, '_PR')
    mod_RPE = add_id_suffix(mod_RPE, '_RPE')

In [7]:
    # add RPE-PR interface reactions and metabolites to RPE and PR models
    from src.combine_RPE_PR import add_interface_RPE, add_interface_PR
    mod_RPE = add_interface_RPE(mod_RPE, 'e_RPE')
    mod_PRrod = add_interface_PR(mod_PRrod, 'e_PR')
    mod_PRcone = add_interface_PR(mod_PRcone, 'e_PR')
    mod_PRrod_Lukowski = add_interface_PR(mod_PRrod_Lukowski, 'e_PR')
    mod_PRcone_Lukowski = add_interface_PR(mod_PRcone_Lukowski, 'e_PR')

In [8]:
    # fuse RPE and PR models   
    from src.modify_model import fix_compartment_dict, add_rxns2groups
    from src.combine_RPE_PR import fuse_RPE_PR, del_RPE_PR_dupl_rxns
    mod_RPE_PRrod = fuse_RPE_PR(mod_RPE.copy(),mod_PRrod)
    mod_RPE_PRcone = fuse_RPE_PR(mod_RPE.copy(),mod_PRcone)
    mod_RPE_PRrod_Lukowski = fuse_RPE_PR(mod_RPE.copy(),mod_PRrod_Lukowski)
    mod_RPE_PRcone_Lukowski = fuse_RPE_PR(mod_RPE.copy(),mod_PRcone_Lukowski)

Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp7xeraim5.lp
Reading time = 0.04 seconds
: 7186 rows, 19642 columns, 65480 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp84d2b5a2.lp
Reading time = 0.04 seconds
: 7186 rows, 19642 columns, 65480 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpcs7fcy8k.lp
Reading time = 0.02 seconds
: 4980 rows, 12484 columns, 46510 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpe90pp0m7.lp
Reading time = 0.03 seconds
: 7186 rows, 19642 columns, 65480 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpg83bh0kx.lp
Reading time = 0.03 seconds
: 7186 rows, 19642 columns, 65480 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpv4lg5_7p.lp
Reading time = 0.03 seconds
: 5055 rows, 12802 columns, 48012 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp11q1bcms.lp
Reading time =

In [9]:
    # fix compartment dictionaries, add reactions to groups, and delete duplicate reactions
    from src.modify_model import fix_compartment_dict, add_rxns2groups
    from src.combine_RPE_PR import del_RPE_PR_dupl_rxns
    mod_RPE_PRrod = fix_compartment_dict(mod_RPE_PRrod)
    mod_RPE_PRrod = add_rxns2groups(mod_RPE_PRrod)
    mod_RPE_PRrod = del_RPE_PR_dupl_rxns(mod_RPE_PRrod)

    mod_RPE_PRcone = fix_compartment_dict(mod_RPE_PRcone)
    mod_RPE_PRcone = add_rxns2groups(mod_RPE_PRcone)    
    mod_RPE_PRcone = del_RPE_PR_dupl_rxns(mod_RPE_PRcone)

    mod_RPE_PRrod_Lukowski = fix_compartment_dict(mod_RPE_PRrod_Lukowski)
    mod_RPE_PRrod_Lukowski = add_rxns2groups(mod_RPE_PRrod_Lukowski)
    mod_RPE_PRrod_Lukowski = del_RPE_PR_dupl_rxns(mod_RPE_PRrod_Lukowski)
    
    mod_RPE_PRcone_Lukowski = fix_compartment_dict(mod_RPE_PRcone_Lukowski)
    mod_RPE_PRcone_Lukowski = add_rxns2groups(mod_RPE_PRcone_Lukowski)
    mod_RPE_PRcone_Lukowski = del_RPE_PR_dupl_rxns(mod_RPE_PRcone_Lukowski)

c:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")
c:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")
c:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")
c:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [10]:
# remove PR exchange reactions from RPE-PR models
for m in [mod_RPE_PRrod, mod_RPE_PRcone, mod_RPE_PRrod_Lukowski, mod_RPE_PRcone_Lukowski]:
    m.remove_reactions([r.id for r in  m.reactions if len(r.products)==0 if 'PR' in r.id]) # remove PR exchange reactions
    m.remove_metabolites([m for m in m.metabolites if len(m.reactions) == 0])              # prune metabolites that are unused
    for r in [r for r in m.reactions if len(r.products) == 0 if '_RPE' in r.id]:           # open efflux for RPE exchange reactions
        r.bounds=(0,1000) 

In [11]:
mod_RPE_PRrod_Lukowski

Name,RPE__VoigtEtAl2019__AverageRodPR_Lukowski
Memory address,14a12bacbe0
Number of metabolites,9738
Number of reactions,12928
Number of genes,2188
Number of groups,149
Objective expression,1.0*MAR13082_RPE - 1.0*MAR13082_RPE_reverse_4a4f4
Compartments,"c_RPE, r_RPE, e_RPE, m_RPE, x_RPE, g_RPE, l_RPE, n_RPE, i_RPE, e_RPE_PR, c_PR, x_PR, m_PR, g_PR, r_PR, l_PR, i_PR"


In [12]:
# save RPE-PR models
from cobra.io import write_sbml_model
write_sbml_model(mod_RPE_PRrod, 'cs_mods/July2023/mod_RPE_PRrod_Liang.xml')
write_sbml_model(mod_RPE_PRcone, 'cs_mods/July2023/mod_RPE_PRcone_Liang.xml')
write_sbml_model(mod_RPE_PRrod_Lukowski, 'cs_mods/July2023/mod_RPE_PRrod_Lukowski.xml')
write_sbml_model(mod_RPE_PRcone_Lukowski, 'cs_mods/July2023/mod_RPE_PRcone_Lukowski.xml')

In [13]:

# create single RPE models with RPE-PR interface reactions
def split_model_create_RPE(RPE_PR_mod):
    """    
    A function to create a single RPE model with RPE-PR interface reactions.

    Parameters
    ----------
    RPE_PR_mod : cobra.core.Model
        RPE-PR model
         
    Returns
    ------- 
    RPE_mod : cobra.core.Model
        RPE model with RPE-PR interface reactions
    """

    m_RPE = RPE_PR_mod.copy()
    m_RPE.remove_reactions([r.id for r in  m_RPE.reactions if len(r.products)==0 if 'PR' in r.id]) # remove PR exchange reactions
    m_RPE.remove_metabolites([m for m in m_RPE.metabolites if len(m.reactions) == 0])              # prune metabolites that are unused
    for r in [r for r in m_RPE.reactions if len(r.products) == 0 if '_RPE' in r

SyntaxError: incomplete input (927882579.py, line 20)

In [ ]:
mod_RPE_PRrod_Lukowski

In [ ]:
    
# seperate RPE-PR models to create single models with a conserved RPE-PR interface    
m.remove_reactions([r.id for r in  m.reactions if len(r.products)==0 if 'PR' in r.id]) # remove PR exchange reactions
m.remove_metabolites([m for m in m.metabolites if len(m.reactions) == 0])              # prune metabolites that are unused